# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure the Azure AI service is configured by following the [configuration steps](../README.md#configure-azure-ai-service-resource).
2. Install the required packages to run this sample.

In [ ]:
%pip install -r ../requirements.txt

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class providing functions to interact with the Content Understanding API. Prior to the official release of the Content Understanding SDK, it serves as a lightweight SDK. Fill in the constants **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, and **AZURE_AI_API_KEY** with your Azure AI Service credentials.

> ⚠️ Important:
Update the sections below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those parts accordingly.
Skipping this step may cause the sample to fail.

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is more secure and highly recommended for production environments.

In [ ]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# For authentication, you can use either token-based auth or subscription key; only one is required
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
# IMPORTANT: Replace with your actual subscription key or set it in your ".env" file if not using token auth
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    # IMPORTANT: Comment out token_provider if using subscription key
    token_provider=token_provider,
    # IMPORTANT: Uncomment this if using subscription key
    # subscription_key=AZURE_AI_API_KEY,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction",  # This header is used for sample usage telemetry; comment out this line if you want to opt out.
)

## Analyzer Templates

This notebook demonstrates field extraction across multiple modalities using Azure AI Content Understanding. We'll walk through each modality step by step:

1. **Document Analysis** - Extract fields from invoices and receipts
2. **Audio Analysis** - Process call recordings and conversational audio
3. **Video Analysis** - Analyze marketing videos and extract insights
4. **Image Analysis** - Extract information from charts and images

Each section will create an analyzer, process sample data, display results, and clean up the analyzer before moving on to the next modality.

### Understanding Analyzer Templates and Schemas

Before creating our first analyzer, it's important to understand the structure and schema of analyzer templates. Custom analyzers in Azure AI Content Understanding are defined using JSON templates that specify which fields to extract and how to process the content.

**Key Schema Components:**

- **`baseAnalyzerId`**: Specifies which prebuilt analyzer to derive from (e.g., `prebuilt-documentAnalyzer`, `prebuilt-audioAnalyzer`, `prebuilt-videoAnalyzer`, `prebuilt-imageAnalyzer`, `prebuilt-callCenter`). This provides foundational capabilities for your custom analyzer.

- **`fields`**: Define specific data points to extract. Each field has:
  - **Field name**: The identifier for the extracted data (required and important for referencing results).
  - **Description**: Optional but helpful for documentation and understanding the field's purpose.
  - **Method**: `"extract"` (for extracting existing information from documents - currently available only for document analysis) or `"generate"` (for generating new insights using AI).

- **`method`**: The overall extraction approach - use `"extract"` for standard field extraction from documents or `"generate"` when leveraging AI to generate insights or summaries.

**Important Note:** The `"extract"` method is currently supported only for document analysis (`prebuilt-documentAnalyzer`). For audio, video, and image modalities, use the `"generate"` method.

Let's take a look at the invoice template to see these concepts in action:

The following is the schema for the document modality used to extract fields from an invoice PDF. This analyzer identifies key invoice elements such as vendor information, amounts, dates, and line items.

In [ ]:
import json

analyzer_template_path = '../analyzer_templates/invoice.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

Below is an example of the schema for the video modality.

In [ ]:
analyzer_template_path = '../analyzer_templates/marketing_video.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

# Document Analysis

Let's begin with document analysis by extracting fields from invoices and receipts. This modality is ideal for processing structured documents and extracting key information like amounts, dates, vendor details, and line items.

## 1. Invoice Field Extraction

Extract fields from an invoice PDF. This analyzer detects essential invoice elements such as vendor information, amounts, dates, and line items.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Invoice Analyzer**

Now, let's create the invoice analyzer and process our sample invoice:

In [ ]:
sample_file_path = '../data/invoice.pdf'
invoice_analyzer_id = "invoice-extraction-" + str(uuid.uuid4())

print(f"Creating invoice analyzer: {invoice_analyzer_id}")
response = client.begin_create_analyzer(invoice_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Invoice analyzer created successfully!")

print(f"Analyzing invoice: {sample_file_path}")
response = client.begin_analyze(invoice_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Invoice Analysis Results**

Let's examine the extracted fields from the invoice:

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Invoice Analyzer**

Clean up the analyzer to manage resources. In production, you would typically keep analyzers for reuse.

In [ ]:
client.delete_analyzer(invoice_analyzer_id)

## 2. Invoice Field Extraction with Source Grounding

Now, let's analyze the same invoice with enhanced field source information and confidence scores. This provides additional context about where each extracted field was found in the document.

**Note**: This custom analyzer is for demonstration purposes only. In production, users should start from `prebuilt-invoice` for invoice processing.

**Key Feature**: This analyzer template uses `estimateFieldSourceAndConfidence: true` in the configuration, which enables the service to provide detailed information about:
- **Field source locations**: Exact coordinates and bounding boxes where each field was found
- **Confidence scores**: The confidence level for each extracted field
- **Enhanced metadata**: Additional context about the extraction process

This is particularly beneficial for applications that need to verify extraction accuracy or provide visual feedback indicating where information was sourced in the document.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice_field_source.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Invoice Field Source Analyzer**

Let's create the enhanced analyzer and process the invoice with source grounding:

In [ ]:
sample_file_path = '../data/invoice.pdf'
invoice_source_analyzer_id = "invoice-field-source-" + str(uuid.uuid4())

print(f"Creating invoice field source analyzer: {invoice_source_analyzer_id}")
response = client.begin_create_analyzer(invoice_source_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Invoice field source analyzer created successfully!")

print(f"Analyzing invoice with field source: {sample_file_path}")
response = client.begin_analyze(invoice_source_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Invoice Field Source Analysis Results**

Review the enhanced results, which include detailed field source locations and confidence scores. Pay special attention to the `confidence` and `source` attributes for each extracted field. The `source` attribute specifies the page number and bounding box coordinates, offering precise context for verification and visualization.

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Invoice Field Source Analyzer**

Clean up the field source analyzer:

In [ ]:
client.delete_analyzer(invoice_source_analyzer_id)

## 3. Receipt Field Extraction

Extract information from a receipt image. This demonstrates how the Content Understanding service can handle different document types and formats.

**Receipt Analyzer Template**

Let's examine the receipt analyzer template:

In [ ]:
analyzer_template_path = '../analyzer_templates/receipt.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Receipt Analyzer**

Now, let's create the receipt analyzer and process our sample receipt:

In [ ]:
sample_file_path = '../data/receipt.png'
receipt_analyzer_id = "receipt-extraction-" + str(uuid.uuid4())

print(f"Creating receipt analyzer: {receipt_analyzer_id}")
response = client.begin_create_analyzer(receipt_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Receipt analyzer created successfully!")

print(f"Analyzing receipt: {sample_file_path}")
response = client.begin_analyze(receipt_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Receipt Analysis Results**

Let's examine the extracted fields from the receipt:

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Receipt Analyzer**

Clean up the receipt analyzer:

In [ ]:
client.delete_analyzer(receipt_analyzer_id)

# Audio Analysis

Now, let's move on to audio analysis. This modality allows extraction of insights from audio files such as call recordings, including summaries, sentiment analysis, topic identification, and entity extraction.

## 4. Call Recording Analytics

Analyze a call center recording to extract insights such as summary, topics discussed, companies mentioned, and people involved in the conversation.

**Call Recording Analytics Template**

Let's examine the call recording analytics template:

In [ ]:
analyzer_template_path = '../analyzer_templates/call_recording_analytics.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

Create and run the call recording analyzer.

In [ ]:
sample_file_path = '../data/callCenterRecording.mp3'
call_analyzer_id = "call-recording-analytics-" + str(uuid.uuid4())

print(f"Creating call recording analyzer: {call_analyzer_id}")
response = client.begin_create_analyzer(call_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Call recording analyzer created successfully!")

print(f"Analyzing call recording: {sample_file_path}")
print("⏳ Note: Audio analysis may take longer than document analysis...")
response = client.begin_analyze(call_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

Call recording analysis results

In [ ]:
print(json.dumps(result_json, indent=2))

Clean up call recording analyzer.

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them.

In [ ]:
client.delete_analyzer(call_analyzer_id)

## 5. Conversational Audio Analytics

Analyze the same audio file focusing on conversational aspects like sentiment analysis and dialogue understanding.

Conversational audio analytics template:

In [ ]:
analyzer_template_path = '../analyzer_templates/conversational_audio_analytics.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

Create and run the conversational audio analyzer.

In [ ]:
sample_file_path = '../data/callCenterRecording.mp3'
conversation_analyzer_id = "conversational-audio-analytics-" + str(uuid.uuid4())

print(f"Creating conversational audio analyzer: {conversation_analyzer_id}")
response = client.begin_create_analyzer(conversation_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Conversational audio analyzer created successfully!")

print(f"Analyzing conversational audio: {sample_file_path}")
print("⏳ Note: Audio analysis may take longer than document analysis...")
response = client.begin_analyze(conversation_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

Conversational audio analysis results

In [ ]:
print(json.dumps(result_json, indent=2))

Clean up conversational audio analyzer.

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them.

In [ ]:
client.delete_analyzer(conversation_analyzer_id)

# Video Analysis

Explore video analysis capabilities. This modality extracts insights from video content including descriptions, sentiment analysis, and key scene identification.

## 6. Marketing Video Analysis

Analyze a marketing video to extract descriptions, sentiment, and key insights valuable for content understanding and marketing analytics.

Marketing video analytics template:

In [ ]:
analyzer_template_path = '../analyzer_templates/marketing_video.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

Create and run the marketing video analyzer.

In [ ]:
sample_file_path = '../data/FlightSimulator.mp4'
video_analyzer_id = "marketing-video-analytics-" + str(uuid.uuid4())

print(f"Creating marketing video analyzer: {video_analyzer_id}")
response = client.begin_create_analyzer(video_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Marketing video analyzer created successfully!")

print(f"Analyzing marketing video: {sample_file_path}")
print("⏳ Note: Video analysis may take significantly longer than document analysis...")
response = client.begin_analyze(video_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

In [ ]:

print(json.dumps(result_json, indent=2))

Clean up marketing video analyzer.

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them.

In [ ]:
client.delete_analyzer(video_analyzer_id)

# Image Analysis

Finally, explore image analysis capabilities. This modality extracts information from charts, diagrams, and other visual content.

## 7. Chart and Image Analysis

Analyze a chart image to extract data points, trends, and insights from visual data representations.

In [ ]:
# Image chart analytics template
analyzer_template_path = '../analyzer_templates/image_chart.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

In [ ]:
sample_file_path = '../data/pieChart.jpg'
chart_analyzer_id = "chart-analysis-" + str(uuid.uuid4())

print(f"Creating chart analyzer: {chart_analyzer_id}")
response = client.begin_create_analyzer(chart_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Chart analyzer created successfully!")

print(f"Analyzing chart: {sample_file_path}")
response = client.begin_analyze(chart_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

In [ ]:
print(json.dumps(result_json, indent=2))

In [ ]:
client.delete_analyzer(chart_analyzer_id)

# Summary

🎉 **Congratulations!** You have successfully explored the major modalities of Azure AI Content Understanding:

✅ **Document Analysis**: Extracted fields from invoices and receipts
✅ **Audio Analysis**: Analyzed call recordings and conversational audio
✅ **Video Analysis**: Processed marketing videos for insights
✅ **Image Analysis**: Extracted information from charts and visual content

## Key Takeaways

- **Multi-modal capabilities**: Content Understanding can process documents, audio, video, and images.
- **Customizable templates**: Each analyzer template is designed for specific use cases but can be customized.
- **Automatic cleanup**: Each analyzer was cleaned up after use to manage resources.
- **Structured output**: All results are returned as consistent JSON for easy integration.

## Next Steps

- Explore the analyzer templates in the `../analyzer_templates/` directory.
- Modify existing templates or create custom ones for your specific use cases.
- Check other notebooks in this repository for advanced scenarios.
- Visit the [Azure AI Content Understanding documentation](https://docs.microsoft.com/azure/ai-services/content-understanding/) for more information.